In [1]:
import polars as pl

bank_a = pl.read_csv("data/bank_a.csv")
bank_b = pl.read_csv("data/bank_b.csv")
bank_c = pl.read_csv("data/bank_c.csv")
for bank in (bank_a,bank_b,bank_c):
    print(bank.columns)

['Transaction ID', 'Date', 'Customer ID', 'Amount', 'Currency', 'Payment Method', 'Status', 'Description']
['transaction_id', 'transaction_date', 'customer_id', 'transaction_amount', 'currency_type', 'payment_method', 'transaction_status', 'transaction_description']
['ID', 'Date of Transaction', 'Customer #', 'Total Amount', 'Currency Used', 'Payment Type', 'Status of Transaction', 'Notes']


In [2]:
from dapter import BaseMapper, accepts


def col_to_date(col: pl.Series):
    return col.str.to_datetime(format='%F')

def clean_str(col:pl.Series):
    return col.str.strip_chars()


date_modifier = (
    accepts("Date", "Date of Transaction", "transaction_date"),
    col_to_date,
)

description_modifier = (
    accepts("transaction_description",'Description','Notes'),
    clean_str
)

class TransactionMapper(BaseMapper):
    transaction_id = accepts("ID", "Transaction ID", "transaction_id")
    date = date_modifier
    amount = accepts("Amount", "Total Amount", "transaction_amount")
    currency = accepts("Currency", "Currency Used", "currency_type")
    description  = description_modifier


mapper = TransactionMapper()

In [3]:
bank_c

ID,Date of Transaction,Customer #,Total Amount,Currency Used,Payment Type,Status of Transaction,Notes
i64,str,i64,f64,str,str,str,str
1,"""2024-08-04""",1011,207.34,"""USD""","""Debit Card""","""Failed""","""Coffee Shop"""
2,"""2024-08-14""",1016,70.5,"""EUR""","""PayPal""","""Completed""","""Streaming Service"""
3,"""2024-08-05""",1011,123.41,"""EUR""","""PayPal""","""Pending""","""Cinema"""
4,"""2024-08-10""",1024,103.02,"""USD""","""Cash""","""Completed""","""Furniture Store"""
5,"""2024-08-14""",1008,247.21,"""GBP""","""PayPal""","""Completed""","""Car Service"""
…,…,…,…,…,…,…,…
46,"""2024-08-01""",1034,216.32,"""EUR""","""Debit Card""","""Pending""","""Flight Booking"""
47,"""2024-08-10""",1027,155.09,"""EUR""","""Credit Card""","""Failed""","""Pet Supplies"""
48,"""2024-08-07""",1002,83.03,"""EUR""","""Debit Card""","""Failed""","""Online Auction"""


In [4]:
mapper.apply(bank_c)

transaction_id,date,amount,currency,description
i64,datetime[μs],f64,str,str
1,2024-08-04 00:00:00,207.34,"""USD""","""Coffee Shop"""
2,2024-08-14 00:00:00,70.5,"""EUR""","""Streaming Service"""
3,2024-08-05 00:00:00,123.41,"""EUR""","""Cinema"""
4,2024-08-10 00:00:00,103.02,"""USD""","""Furniture Store"""
5,2024-08-14 00:00:00,247.21,"""GBP""","""Car Service"""
…,…,…,…,…
46,2024-08-01 00:00:00,216.32,"""EUR""","""Flight Booking"""
47,2024-08-10 00:00:00,155.09,"""EUR""","""Pet Supplies"""
48,2024-08-07 00:00:00,83.03,"""EUR""","""Online Auction"""
